In [1]:
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from load_data_and_distribution_analisys import split_dataset_v1, load_dataset_cleaned

/Users/marcodeano/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/marcodeano/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Random forest con feature reduction e cross validation

In questa ultima parte relativa l'utilizzo della RandomForest come modello di appredimento, useremo la cross validation per valutare in maniera più precisa le performance dopo aver fatto feature selection.
Siccome nello step precedente abbiamo allenato il nostro modello tenendo però costante il set di train e di validazione, potremmo aver "overfittato" in base alla suddivisione specifica; con la cross-validation, testiamo il modello su diverse porzioni del dataset e possiamo valutare se le feature scelte migliorano davvero il modello.

In [2]:
dataset_cleaned = load_dataset_cleaned()

Path to dataset files: /Users/marcodeano/.cache/kagglehub/datasets/sooyoungher/smoking-drinking-dataset/versions/2
   sex  age  height  weight  waistline  sight_left  sight_right  hear_left  \
0    1   35     170      75       90.0         1.0          1.0        1.0   
1    1   30     180      80       89.0         0.9          1.2        1.0   
2    1   40     165      75       91.0         1.2          1.5        1.0   
3    1   50     175      80       91.0         1.5          1.2        1.0   
4    1   50     165      60       80.0         1.0          1.2        1.0   

   hear_right    SBP  ...  LDL_chole  triglyceride  hemoglobin  urine_protein  \
0         1.0  120.0  ...      126.0          92.0        17.1            1.0   
1         1.0  130.0  ...      148.0         121.0        15.8            1.0   
2         1.0  120.0  ...       74.0         104.0        15.8            1.0   
3         1.0  145.0  ...      104.0         106.0        17.6            1.0   
4         1

In [5]:
X_smoke, Y_smoke, X_drink, Y_drink = split_dataset_v1(dataset_cleaned)

In [4]:
best_selected_features_d = ["sex", "age", "gamma_GTP", "HDL_chole", "SGOT_ALT", "weight", "SGOT_AST", "tot_chole", "triglyceride", "LDL_chole", "serum_creatinine", "DBP", "hemoglobin", "BLDS", "hear_right", "waistline", "hear_left", "height"]
best_selected_features_s = ["age", "sex", "height", "weight", "hear_right", "sight_right", "DBP", "BLDS", "HDL_chole", "LDL_chole", "triglyceride", "serum_creatinine", "SGOT_AST", "SGOT_ALT", "gamma_GTP", "hemoglobin"]

In [ ]:
forest = RandomForestClassifier(n_estimators=100, max_depth=20, min_samples_split=50, random_state=42, class_weight="balanced")
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
f1_score_rf_s = cross_val_score(forest, X_smoke.loc[:, best_selected_features_s], Y_smoke, cv=cv, scoring='f1_weighted', n_jobs=-1)
accuracy_rf_s = cross_val_score(forest, X_smoke.loc[:, best_selected_features_s], Y_smoke, cv=cv, scoring='balanced_accuracy', n_jobs=-1)
print(f"F1-score medio: {f1_score_rf_s.mean()*100:.2f}%")
print(f"Accuratezza media: {accuracy_rf_s.mean()*100:.2f}%")

F1-score medio: 70.42%
Accuratezza media: 65.02%


In [ ]:
forest = RandomForestClassifier(n_estimators=100, max_depth=20, min_samples_split=50, random_state=42)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
f1_score_rf_d = cross_val_score(forest, X_drink.loc[:, best_selected_features_d], Y_drink, cv=cv, scoring='f1', n_jobs=-1)
accuracy_rf_d = cross_val_score(forest, X_drink.loc[:, best_selected_features_d], Y_drink, cv=cv, scoring='accuracy', n_jobs=-1)
print(f"F1-score medio: {f1_score_rf_d.mean()*100:.2f}%")
print(f"Accuratezza media: {accuracy_rf_d.mean()*100:.2f}%")

F1-score medio: 73.56%
Accuratezza media: 73.11%
